In [1]:
!pip install transformers > /dev/null
!pip install datasets > /dev/null

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric, load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import EarlyStoppingCallback
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [3]:
df = pd.read_csv("data/ArSenTD-Lev/ArSenTD-LEV.tsv", sep="\t")
df

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك
...,...,...,...,...,...,...
3995,صلاح من لاعب في المقاولون العرب يحلم ان يلعب ل...,lebanon,sports,very_positive,explicit,بطل قومي
3996,الملك سلمان بن عبد العزيز: تطبيق الأنظمة بحزم ...,lebanon,politics,positive,explicit,أساء استغلال السلطة والنفوذ
3997,@ZahraaIraq9 😂 كل ما ادخل حسابي الكه تغريداتج ...,jordan,personal,negative,explicit,تغريداتج عن حب العراق
3998,شو هالشعب نحنا اللي عايش بلا مي وكهربا والزبال...,lebanon,politics,very_negative,explicit,عايش بلا مي وكهربا


In [4]:
set(df['Sentiment'])

{'negative', 'neutral', 'positive', 'very_negative', 'very_positive'}

In [5]:
len(df[df['Sentiment']=='very_negative'])

630

In [6]:
len(df[df['Sentiment']=='very_positive'])

397

In [7]:
df.drop(['Country', 'Topic', 'Sentiment_Expression', 'Sentiment_Target'], axis=1, inplace=True)
df.head()

,Tweet,Sentiment
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",negative
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,positive
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,neutral
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,negative
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,negative


In [8]:
df.isna().sum()

Tweet        0
Sentiment    0
dtype: int64

In [9]:
df.rename(columns={'Tweet': 'text', 'Sentiment': 'label'}, inplace=True)
df

,text,label
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",negative
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,positive
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,neutral
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,negative
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,negative
...,...,...
3995,صلاح من لاعب في المقاولون العرب يحلم ان يلعب ل...,very_positive
3996,الملك سلمان بن عبد العزيز: تطبيق الأنظمة بحزم ...,positive
3997,@ZahraaIraq9 😂 كل ما ادخل حسابي الكه تغريداتج ...,negative
3998,شو هالشعب نحنا اللي عايش بلا مي وكهربا والزبال...,very_negative


In [10]:
values = ['very_positive', 'very_negative']
df = df[df.label.isin(values) == False]
df

,text,label
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",negative
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,positive
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,neutral
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,negative
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,negative
...,...,...
3992,@andolini74 الصوت جميل ....بس - ومع اني مش خبي...,negative
3994,الجو غداً غائم جزئيا مع انخفاض بسيط بدرجات الح...,neutral
3996,الملك سلمان بن عبد العزيز: تطبيق الأنظمة بحزم ...,positive
3997,@ZahraaIraq9 😂 كل ما ادخل حسابي الكه تغريداتج ...,negative


In [11]:
df['label'].value_counts(normalize=True)

negative    0.421460
neutral     0.297679
positive    0.280861
Name: label, dtype: float64

In [12]:
def change_label(label):
    if label== 'negative':
        return 0
    elif label=='positive':
        return 1
    else:
        return 2

df['label'] = df['label'].apply(change_label)
df.head()   


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,text,label
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",0
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,1
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,2
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,0
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,0


In [13]:
X = df['text']
y = df['label']

# create train test datasets
train_tmp_X, test_X, train_tmp_y, test_y = train_test_split(X, y,   test_size=0.1, random_state=43, stratify=y)
# create validation dataset
train_X, val_X, train_y, val_y = train_test_split(train_tmp_X, train_tmp_y, test_size=0.1, random_state=43, stratify=train_tmp_y)

In [14]:
len(train_y), len(val_y) ,len(test_y)

(2407, 268, 298)

In [15]:
frame_train = { 'text': train_X, 'label': train_y}
df_train = pd.DataFrame(frame_train)
df_train.to_csv("data/ArSenTD-Lev/train.csv", encoding='utf-8', index=False)

#df val
frame_val = { 'text': val_X, 'label': val_y}
df_val = pd.DataFrame(frame_val)
df_val.to_csv("data/ArSenTD-Lev/val.csv", encoding='utf-8', index=False)


#df test
frame_test = { 'text': test_X, 'label': test_y }
df_test = pd.DataFrame(frame_test)
df_test.to_csv("data/ArSenTD-Lev/test.csv", encoding='utf-8', index=False)

In [16]:
df_test.shape

(298, 2)

In [17]:
ds = load_dataset('csv', data_files={'train': 'data/ArSenTD-Lev/train.csv', 'val': 'data/ArSenTD-Lev/val.csv', 'test': 'data/ArSenTD-Lev/test.csv'})
ds

Using custom data configuration default-ca0dab91dba25b2c


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ca0dab91dba25b2c/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2407
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 268
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 298
    })
})

In [18]:
model_name = "mofawzy/bert-arsentd-lev"
model_testing = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)
tokenizer_testing = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/903 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/733k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [19]:
def tokenize(examples):
    outputs = tokenizer_testing(examples['text'], truncation=True,max_length=128)
    return outputs

tokenized_ds = ds.map(tokenize, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
test_trainer = Trainer(model=model_testing,tokenizer=tokenizer_testing)
raw_pred, _, _ = test_trainer.predict(tokenized_ds['test'])
y_pred = np.argmax(raw_pred, axis=1)
print(classification_report(test_y, y_pred,digits=4))

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 298
  Batch size = 8


              precision    recall  f1-score   support

           0     0.8211    0.8080    0.8145       125
           1     0.7174    0.7857    0.7500        84
           2     0.6867    0.6404    0.6628        89

    accuracy                         0.7517       298
   macro avg     0.7418    0.7447    0.7424       298
weighted avg     0.7518    0.7517    0.7510       298

